## Libraries and Modules

The following libraries and modules are imported at the beginning of the code:

- `numpy`: a library for numerical computing in Python
- `pandas`: a library for data manipulation and analysis
- `os`: a module for interacting with the operating system
- `glob`: a module for finding all the pathnames matching a specified pattern
- `time`: a module for working with time-related functions
- `copy`: a module for creating copies of objects
- `random`: a module for generating random numbers
- `zipfile`: a module for working with ZIP archives
- `matplotlib.pyplot`: a module for creating visualizations in Python
- `PIL.Image`: a module for working with images in Python
- `torch`: a library for machine learning in Python
- `torch.nn`: a module for building neural networks in PyTorch
- `torch.optim`: a module for optimizing neural networks in PyTorch
- `torch.nn.functional`: a module for applying various functions to tensors in PyTorch
- `torchvision`: a library for computer vision tasks in PyTorch
- `torchvision.models`: a module containing pre-trained models for computer vision tasks in PyTorch
- `torch.utils.data`: a module for working with datasets in PyTorch


In [ ]:
%matplotlib inline

# Importing necessary libraries and modules
import numpy as np # for numerical computing
import pandas as pd # for manipulation and analysis
import os, glob, time, copy, random, zipfile # for interacting with the operating system and working with files
import matplotlib.pyplot as plt # for creating visualizations
from PIL import Image # for working with images

import torch # for machine learning
import torch.nn as nn # for building neural networks in PyTorch
import torch.optim as optim # for optimizing neural networks in PyTorch
import torchvision # for computer vision tasks in PyTorch
from torchvision import models, transforms # for loading and preprocessing image data
import torchvision.models as models # for using pre-trained models in PyTorch
from torch.utils.data import DataLoader, Dataset # for working with datasets in PyTorch
import tqdm


## Setup

The code begins by creating a new directory to store the data using the `os.mkdir()` function. It then defines the paths for the training and testing data using variables.

The code then uses the `zipfile` module to extract the training and testing data from the ZIP files. The `with` statement is used to ensure that the ZIP files are properly closed after the data has been extracted.

Finally, the code uses the `glob` module to get the file paths for the training and testing images. The `glob.glob()` function is used to find all files in the specified directory that match the specified pattern. The device(ie. GPU/CPU) which we will be using for the model is then defined


In [ ]:
# Creating a new directory to store the data
os.mkdir('../data')

# Defining the paths for the training and testing data
base_dir = '../input/dogs-vs-cats-redux-kernels-edition'
train_dir = '../data/train'
test_dir = '../data/test'

# Extracting the training and testing data from the ZIP files
with zipfile.ZipFile(os.path.join(base_dir, 'train.zip')) as train_zip:
    train_zip.extractall('../data')
    
with zipfile.ZipFile(os.path.join(base_dir, 'test.zip')) as test_zip:
    test_zip.extractall('../data')

# Getting the file paths for the training and testing images
train_x = glob.glob(os.path.join(train_dir,'*.jpg'))
test_x = glob.glob(os.path.join(test_dir,'*.jpg'))


# Set the device which we will be using for the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## Dataset Setup

The code above defines a custom dataset class for loading and preprocessing image data. It includes several functions for working with image files and applying transformations to the images.


The `manualDataset` class is defined with several parameters, including the file paths for the images, the split between training and testing data, and whether or not to use a validation split. The class includes functions for loading and preprocessing the image data, including applying transformations to the images and extracting the labels from the file names.

The `__init__()` function initializes the class with the specified parameters and defines the transformations to be applied to the images. The `__len__()` function returns the length of the dataset, and the `__getitem__()` function loads and preprocesses the image data and returns it along with the corresponding label.

In [ ]:
# Defining a custom dataset class for loading and preprocessing image data
class manualDataset(Dataset):
    def __init__(self, files, split='train', val_split=False):
        self.raw_files = files
        self.split = split
        self.val_split = val_split
        
        # Defining the transformations to be applied to the images
        self.train_transform = transforms.Compose([transforms.Resize((224,224)), transforms.RandomHorizontalFlip(), transforms.ToTensor()])
        self.test_transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
      
        # Splitting the data into training and testing sets
        if split == 'test':
            if val_split:
                self.raw_files = self.raw_files[:10]
            else:
                self.raw_files = self.raw_files[10:]
        
    def __len__(self):
        return len(self.raw_files)
    
    def __getitem__(self, idx):
        # Reading in the image file
        raw_file = self.raw_files[idx]
        raw = Image.open(raw_file)
        
        # Applying the appropriate transformation based on the split
        if self.split == 'train':
            raw = self.train_transform(raw)
        elif self.split == 'test':
            raw = self.test_transform(raw)
        
        # Extracting the label from the file name
        label = raw_file.split('/')[-1].split('.')[0]
        if label == 'dog':
            label = 1
        elif label == 'cat':
            label = 0
        
        return raw, label


## Datasets and Dataloaders Setup


The code creates instances of the `manualDataset` class for the training, testing, and validation data. The `split` parameter is used to specify whether the data is for training or testing, and the `val_split` parameter is used to specify whether to use a validation split.

The code then defines the batch sizes for the training and testing data, and creates instances of the `DataLoader` class for the training, testing, and validation data. The `batch_size` parameter is used to specify the number of samples per batch, and the `shuffle` parameter is used to specify whether to shuffle the training data.



In [ ]:
# Creating instances of the manualDataset class for the training, testing, and validation datasets
train_dataset = manualDataset(train_x, split='train')
test_dataset = manualDataset(test_x, split='test')
val_dataset = manualDataset(train_x, split='test', val_split=True)

# Defining the batch sizes for the training and testing data
train_batch_size = 32
test_batch_size = 1

# Creating instances of the DataLoader class for the training, testing, and validation data
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size)
val_loader = DataLoader(val_dataset, batch_size=test_batch_size)


## Transfer Learning using Resnet

The code begins by loading the ResNet50 model with pre-trained weights using the `models.resnet50()` function. It then modifies the last fully connected layer (fc) to customize the model for binary classification.

The code defines the number of features in the last layer using the `resnet50.fc.in_features` attribute, and creates a custom classifier using the `nn.Sequential()` function. The custom classifier includes two linear layers with ReLU activation functions and a final sigmoid activation function.

The code attaches the custom classifier to the ResNet50 model using the `resnet50.fc = custom_classifier` statement. It then moves the model to the appropriate device (GPU or CPU) using the `torch.device()` function.

In [ ]:

resnet50 = models.resnet50(pretrained=True)


    

# Remove the last fully connected layer (fc) to customize the model
num_features = resnet50.fc.in_features
resnet50.fc = nn.Sequential()

# Add custom layers for binary classification
custom_classifier = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),

    nn.Linear(512, 1),
    nn.Sigmoid()
)

# Attach the custom classifier to the ResNet50 model
resnet50.fc = custom_classifier

# Move the model to the appropriate device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet50.to(device)


## Freezing Untouched Layers of Resnet


The code begins by defining an empty list called `param_to_optimize` to store the parameters of the custom classifier that will be optimized. It then iterates through the named parameters of the ResNet50 model using the `resnet50.named_parameters()` function.

For each parameter, the code checks if the name contains the string 'fc' using the `if('fc' in name)` statement. If the name contains 'fc', the `requires_grad` attribute is set to `True` for the parameter and the parameter is added to the `param_to_optimize` list. If the name does not contain 'fc', the `requires_grad` attribute is set to `False' for the parameter.


In [ ]:
param_to_optimize=[]
for name, param in resnet50.named_parameters():
    if('fc' in name): 
        param.requires_grad = True
        param_to_optimize.append(param)
    else:
        param.requires_grad = False

## Model Setup & Training

The code begins by defining the loss function and optimizer for training the model using `nn.BCELoss()` and `optim.SGD()`, respectively. The `params` parameter is used to specify the parameters to optimize, which are the parameters of the custom classifier defined in the previous code block.

The number of epochs to train the model is defined using the `epochs` variable. The model is then set to training mode using `resnet50.train()`.

The model is trained for the specified number of epochs using a nested loop. The outer loop iterates through the epochs, and the inner loop iterates through the training data using the `train_loader` DataLoader.

For each batch of data, the data and labels are moved to the appropriate device (GPU or CPU) using `to()`. The output of the model is computed using `resnet50()`, the loss is calculated using `nn.BCELoss()`, and backpropagation is performed using `backward()`. The `optimizer.step()` function is used to update the model parameters.

The epoch loss and accuracy are computed for each epoch using `epoch_loss` and `epoch_accuracy`. The epoch number and epoch loss are printed using `print()`.

After training the model for each epoch, the model is set to evaluation mode using `resnet50.eval()`. The validation data is then iterated through using the `val_loader` DataLoader object.

For each batch of data, the output of the model is computed using `resnet50()`, the validation loss is calculated using `nn.BCELoss()`, and the epoch validation loss is computed using `epoch_val_loss`. The epoch number and epoch validation loss are printed using `print()`.



In [ ]:
# Defining the loss function and optimizer for training the model
criterion = nn.BCELoss()
optimizer = optim.SGD(params=param_to_optimize, lr=0.001, momentum=0.9)

# Defining the number of epochs to train the model
epochs = 10

# Setting the model to training mode
resnet50.train()

# Training the model for the specified number of epochs
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    
    # Iterating through the training data
    for data, label in tqdm.tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)
        label = label.type(torch.float)
        output = resnet50(data)
        label = label.unsqueeze(1)
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += (loss.item())/len(train_loader)
        
    print('Epoch : {},  train loss : {}'.format(epoch+1,epoch_loss))
    
    # Setting the model to evaluation mode
    resnet50.eval()
    with torch.no_grad():
        epoch_val_accuracy=0
        epoch_val_loss =0
        
        # Iterating through the validation data
        for data, label in val_loader:
            data = data.to(device)
            label = label.to(device)
            label = label.type(torch.float)
            output = resnet50(data)
            label = label.unsqueeze(1)
            
            val_output = resnet50(data)
            val_loss = criterion(val_output,label)
            
            epoch_val_loss += val_loss/ len(val_loader)
            
        print('Epoch : {} , val_loss : {}'.format(epoch+1,epoch_val_loss))


## Submission

Finally, we evaluate it on the test dataset for submission

The code begins by creating empty lists to store the IDs and predictions. The model is then set to evaluation mode using `resnet50.eval()`.

Gradient calculation is disabled to save memory using the `torch.no_grad()` statement. The code iterates through the test data using a `for` loop and opens each image using `Image.open()`. The ID is extracted from the file path using string manipulation.

The test transformations are applied to the image using `transforms.Compose()` and the output is computed using `resnet50()`. The output is converted to a list using `tolist()` and added to the `pred_list` list along with the ID.

The prediction list is converted to a numpy array using `np.array()` and rounded to the nearest integer using `np.round()` and `astype(int)`.

A DataFrame is created with the IDs and predictions using `pd.DataFrame()`. The DataFrame is sorted by ID using `sort_values()` and the index is reset using `reset_index()`. Finally, the DataFrame is saved to a CSV file named 'submission.csv' using `to_csv()`.


In [ ]:
# Creating empty lists to store the IDs and predictions
id_list = []
pred_list = []

# Setting the model to evaluation mode
resnet50.eval()

# Disabling gradient calculation to save memory
with torch.no_grad():
    # Iterating through the test data
    for test_path in (test_x):
        # Opening the image and extracting the ID
        img = Image.open(test_path)
        _id = int(test_path.split('/')[-1].split('.')[0])
        
        # Applying the test transformations to the image
        test_transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor() ])
        img = test_transform(img)
        img = img.unsqueeze(0)
        img = img.to(device)

        # Computing the output of the model and converting it to a list
        outputs = (resnet50(img)).tolist()
        
        # Adding the ID and prediction to the respective lists
        id_list.append(_id)
        pred_list.append(outputs[0])
    
# Converting the prediction list to a numpy array and rounding it to the nearest integer
pred_list = np.array(pred_list)
pred_list = np.round(pred_list.flatten()).astype(int)

# Creating a DataFrame with the IDs and predictions
res = pd.DataFrame({
    'id': id_list,
    'label': pred_list
})

# Sorting the DataFrame by ID and resetting the index
res.sort_values(by='id', inplace=True)
res.reset_index(drop=True, inplace=True)

# Saving the DataFrame to a CSV file
res.to_csv('submission.csv', index=False)
